In [1]:
import pandas as pd
import duckdb
import os
import os, json
from uuid import uuid4
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer

2025-07-23 15:18:01.297135: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 15:18:01.306734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753276681.318245 3603422 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753276681.321435 3603422 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753276681.330022 3603422 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Load LaBSE model and push to GPU
model = SentenceTransformer("sentence-transformers/LaBSE")
device = "cuda"  # ensure GPU is used

# Encode in batches
batch_size = 256
model_name = "LaBSE"

In [11]:
import torch
print(torch.cuda.max_memory_allocated() / 1e9, "GB")

3.772739584 GB


In [3]:
conn = duckdb.connect("/srv/data/grela_v0-2.duckdb")

In [4]:
# Query to get table and column information
query = """
    SELECT
        table_name,
        column_name,
        data_type,
        is_nullable,
        column_default
    FROM information_schema.columns
    ORDER BY table_name, ordinal_position
"""

# Execute the query and fetch the schema information as a DataFrame
df = conn.execute(query).fetchdf()

# Group the schema details by table
tables = df.groupby("table_name")

# Markdown generation
markdown = "# Database Schema Documentation\n\n"
for table_name, group in tables:
    markdown += f"## Table: `{table_name}`\n\n"
    markdown += "| Column Name     | Data Type    | Is Nullable | Default Value |\n"
    markdown += "|-----------------|-------------|-------------|---------------|\n"

    for _, row in group.iterrows():
        markdown += (
            f"| {row['column_name']} | {row['data_type']} | "
            f"{row['is_nullable']} | {row['column_default'] or 'N/A'} |\n"
        )

    markdown += "\n"  # Add a space between tables

In [5]:
print(markdown)

# Database Schema Documentation

## Table: `sentence_embeddings`

| Column Name     | Data Type    | Is Nullable | Default Value |
|-----------------|-------------|-------------|---------------|
| sentence_id | VARCHAR | NO | N/A |
| grela_id | VARCHAR | YES | N/A |
| model | VARCHAR | YES | N/A |
| embedding | JSON | YES | N/A |

## Table: `sentences`

| Column Name     | Data Type    | Is Nullable | Default Value |
|-----------------|-------------|-------------|---------------|
| sentence_id | VARCHAR | YES | N/A |
| grela_id | VARCHAR | YES | N/A |
| position | INTEGER | YES | N/A |
| text | VARCHAR | YES | N/A |
| subwork_id | VARCHAR | YES | N/A |

## Table: `tokens`

| Column Name     | Data Type    | Is Nullable | Default Value |
|-----------------|-------------|-------------|---------------|
| sentence_id | VARCHAR | YES | N/A |
| grela_id | VARCHAR | YES | N/A |
| token_text | VARCHAR | YES | N/A |
| lemma | VARCHAR | YES | N/A |
| pos | VARCHAR | YES | N/A |
| char_start | IN

In [39]:
# Create table for embeddings (if not exists)
conn.execute("""
CREATE TABLE IF NOT EXISTS sentence_embeddings (
    sentence_id TEXT PRIMARY KEY,
    grela_id TEXT,
    model TEXT,
    embedding JSON
)
""")

In [40]:
# Load all Vulgate sentences
query = """
SELECT sentence_id, grela_id, text
FROM sentences
WHERE grela_id LIKE 'vulgate_%'
"""
df = conn.execute(query).fetchdf()

In [41]:
len(df)

35254

In [43]:
%%time
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    print(f"Encoding batch {i}–{i + len(batch)}")

    # Encode with GPU
    embeddings = model.encode(batch["text"].tolist(), convert_to_numpy=True, device=device)

    # Insert into DuckDB
    for j, row in batch.iterrows():
        emb_vector = embeddings[j - i].tolist()
        conn.execute("""
            INSERT OR REPLACE INTO sentence_embeddings (sentence_id, grela_id, model, embedding)
            VALUES (?, ?, ?, ?)
        """, (row["sentence_id"], row["grela_id"], model_name, json.dumps(emb_vector)))

Encoding batch 0–256
Encoding batch 256–512
Encoding batch 512–768
Encoding batch 768–1024
Encoding batch 1024–1280
Encoding batch 1280–1536
Encoding batch 1536–1792
Encoding batch 1792–2048
Encoding batch 2048–2304
Encoding batch 2304–2560
Encoding batch 2560–2816
Encoding batch 2816–3072
Encoding batch 3072–3328
Encoding batch 3328–3584
Encoding batch 3584–3840
Encoding batch 3840–4096
Encoding batch 4096–4352
Encoding batch 4352–4608
Encoding batch 4608–4864
Encoding batch 4864–5120
Encoding batch 5120–5376
Encoding batch 5376–5632
Encoding batch 5632–5888
Encoding batch 5888–6144
Encoding batch 6144–6400
Encoding batch 6400–6656
Encoding batch 6656–6912
Encoding batch 6912–7168
Encoding batch 7168–7424
Encoding batch 7424–7680
Encoding batch 7680–7936
Encoding batch 7936–8192
Encoding batch 8192–8448
Encoding batch 8448–8704
Encoding batch 8704–8960
Encoding batch 8960–9216
Encoding batch 9216–9472
Encoding batch 9472–9728
Encoding batch 9728–9984
Encoding batch 9984–10240
Encoding

In [7]:

# Load all register sentences
query = """
SELECT sentence_id, grela_id, text
FROM sentences
WHERE grela_id LIKE 'cc_10265'
"""
df = conn.execute(query).fetchdf()

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    print(f"Encoding batch {i}–{i + len(batch)}")

    # Encode with GPU
    embeddings = model.encode(batch["text"].tolist(), convert_to_numpy=True, device=device)

    # Insert into DuckDB
    for j, row in batch.iterrows():
        emb_vector = embeddings[j - i].tolist()
        conn.execute("""
            INSERT OR REPLACE INTO sentence_embeddings (sentence_id, grela_id, model, embedding)
            VALUES (?, ?, ?, ?)
        """, (row["sentence_id"], row["grela_id"], model_name, json.dumps(emb_vector)))

Encoding batch 0–256
Encoding batch 256–512
Encoding batch 512–768
Encoding batch 768–1024
Encoding batch 1024–1280
Encoding batch 1280–1536
Encoding batch 1536–1792
Encoding batch 1792–2048
Encoding batch 2048–2304
Encoding batch 2304–2560
Encoding batch 2560–2816
Encoding batch 2816–3072
Encoding batch 3072–3328
Encoding batch 3328–3584
Encoding batch 3584–3840
Encoding batch 3840–4096
Encoding batch 4096–4352
Encoding batch 4352–4608
Encoding batch 4608–4864
Encoding batch 4864–5120
Encoding batch 5120–5376
Encoding batch 5376–5399


In [46]:
query = """
        SELECT
            s.grela_id,
            s.sentence_id,
            s.text
        FROM sentences s
        JOIN works w ON s.grela_id = w.grela_id
        WHERE (w.not_before > 1000 AND w.not_before < 1100)
           OR (w.not_after > 1000 AND w.not_after < 1100);
"""
df = conn.execute(query).fetchdf()

In [47]:
len(df)

1357091

In [52]:
%%time
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    # Encode with GPU
    embeddings = model.encode(batch["text"].tolist(), convert_to_numpy=True, device=device)

    # Insert into DuckDB
    for j, row in batch.iterrows():
        emb_vector = embeddings[j - i].tolist()
        conn.execute("""
            INSERT OR REPLACE INTO sentence_embeddings (sentence_id, grela_id, model, embedding)
            VALUES (?, ?, ?, ?)
        """, (row["sentence_id"], row["grela_id"], model_name, json.dumps(emb_vector)))

CPU times: user 3h 11min 26s, sys: 1h 53min 47s, total: 5h 5min 14s
Wall time: 5h 10min 44s


In [8]:
conn.close()